In [3]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
from nilearn import image, masking, plotting, glm
from nilearn.input_data import NiftiMasker
from nilearn.glm.first_level import FirstLevelModel

In [35]:
# Define data paths
data_dir = "/jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep" 
subjects = ["sub-01/func/", "sub-03/func/", "sub-04/func/"]  # Assuming 20 subjects, update as needed
task_prefix = "imagine"

In [36]:
output_dir = "neural_activity"
os.makedirs(output_dir, exist_ok=True)

def extract_neural_activity(subject):
    print(f"Processing {subject}...")

    # Find the preprocessed BOLD image
    bold_file = None
    for f in os.listdir(os.path.join(data_dir, subject)):
        if task_prefix in f and "desc-preproc_bold.nii.gz" in f:
            bold_file = os.path.join(data_dir, subject, f)
            break

    if not bold_file:
        print(f"No preprocessed BOLD file found for {subject}.")
        return

    # Load fMRI data
    bold_img = nib.load(bold_file)

    # Find brain mask
    mask_file = None
    for f in os.listdir(os.path.join(data_dir, subject)):
        if task_prefix in f and "desc-brain_mask.nii.gz" in f:
            mask_file = os.path.join(data_dir, subject, f)
            break

    if mask_file:
        mask_img = nib.load(mask_file)
        print(f"Using brain mask: {mask_file}")
    else:
        mask_img = masking.compute_epi_mask(bold_img)  # Compute mask if not provided
        print(f"Computed brain mask for {subject}.")

    # Extract time series from masked brain voxels
    masker = NiftiMasker(mask_img=mask_img, standardize=True)
    time_series = masker.fit_transform(bold_img)  # (timepoints x voxels)
    
    print(f"Extracted neural activity shape: {time_series.shape}")  # (T, N_voxels)



In [37]:
# Run processing for all subjects
for subject in subjects:
    extract_neural_activity(subject)

Processing sub-01/func/...
Using brain mask: /jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep/sub-01/func/sub-01_task-imagine_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz


/usr/people/yx7967/miniconda3/envs/myenv/lib/python3.8/site-packages/nilearn/maskers/nifti_masker.py:110: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


Extracted neural activity shape: (535, 73291)
Processing sub-03/func/...
Using brain mask: /jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep/sub-03/func/sub-03_task-imagine_space-T1w_desc-brain_mask.nii.gz
Extracted neural activity shape: (535, 60764)
Processing sub-04/func/...
Using brain mask: /jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep/sub-04/func/sub-04_task-imagine_space-T1w_desc-brain_mask.nii.gz
Extracted neural activity shape: (535, 56978)
